In [603]:
global_namespace = Dict(":y" => "x->x + 3")

function metajulia_repl()
    print(global_namespace)
    print(">>", " ")
    ans = readline()
    print("ans: $(ans)\n")
    parsed_ans = Meta.parse(ans)
    my_eval(parsed_ans, global_namespace)
end

function my_eval(expr, namespace)
    if !(expr isa Expr)
        if expr isa Symbol
            return namespace[expr]
        else
            return expr
        end
    else
        if expr.head == :block
            for arg in expr.args[begin:end-1]
                if !(arg isa LineNumberNode)
                    my_eval(arg, namespace)
                end
            end
            return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], namespace) : nothing
        elseif expr.head == :&&
            return reduce(&, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :||
            return reduce(|, [my_eval(expr.args[x], namespace) for x in 1:length(expr.args)])
        elseif expr.head == :if
            return my_eval(expr.args[1], namespace) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :elseif
            return my_eval(expr.args[1].args[2]) ? my_eval(expr.args[2], namespace) : my_eval(expr.args[3], namespace)
        elseif expr.head == :(=)
            if (expr.args[1] isa Expr)
                println("arrived at function definition!")
                namespace[expr.args[1].args[1]] = string("(", expr.args[1].args[2], "->",  expr.args[2].args[end], ")")
            else
                namespace[expr.args[1]] = my_eval(expr.args[2], namespace)
            end
        elseif expr.head == :let
            # println("let")
            # println(expr.args[begin])
            # println(expr.args[end])
            let
                scope_dict = copy(namespace)
                for arg in expr.args[begin:end-1]
                    if !(arg isa LineNumberNode)
                        my_eval(arg, scope_dict)
                    end
                end
                # println(!(expr.args[end] isa LineNumberNode))
                return !(expr.args[end] isa LineNumberNode) ? my_eval(expr.args[end], scope_dict) : nothing
            end
        elseif expr.head == :call
            if expr.args[1] == :+
                return reduce(+, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :*
                return reduce(*, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :/
                return reduce(/, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :<
                return reduce(<, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            elseif expr.args[1] == :>
                return reduce(>, [my_eval(expr.args[x], namespace) for x in 2:length(expr.args)])
            end
        end
    end
end

my_eval (generic function with 2 methods)

In [604]:
metajulia_repl()

Dict(":y" => "x->x + 3")>> 

stdin>  y(3)


longer than 3!
modifying ans
ans:	:y
x->x + 3
ans: nothing(3)


In [528]:
Meta.parse("y -> y+2").args[2].args[end].args[2] == Meta.parse("y -> y+2").args[1]

true

In [516]:
(y -> y+2)(3)

5

In [512]:
Meta.parse(global_namespace[:x])(4)

LoadError: MethodError: objects of type Expr are not callable

In [574]:
Meta.parse("x(y)").args[1].

LoadError: type Symbol has no field name

In [246]:
my_eval()

LoadError: MethodError: no method matching my_eval()

[0mClosest candidates are:
[0m  my_eval([91m::Any[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[242]:9[24m[39m


In [490]:
Meta.parse("x(y) = y*y+34").args[2].args[end]


:(y * y + 34)

In [333]:
Meta.parse("let var = 100 end").args[end].args

1-element Vector{Any}:
 :(#= none:1 =#)

In [547]:
Meta.parse(global_namespace[:x])

LoadError: MethodError: objects of type String are not callable

In [228]:
typeof(x)

Int64

In [480]:
myfun = y -> quote
    #= none:1 =#
    y + 1
end

#772 (generic function with 1 method)

In [481]:
myfun(3)

quote
    #= In[480]:3 =#
    y + 1
end

In [484]:
my_eval(quote
    #= none:1 =#
    y + 1
end)

LoadError: KeyError: key :y not found

In [535]:
Meta.parse("(y->y+1)(3)").args[1]

:(y->begin
          #= none:1 =#
          y + 1
      end)

## 